In [15]:
from random import random
from tqdm.auto import tqdm

# Mutable state
variables = {
    'prey_population': 0.5,
    'predator_population': 0.5
}

# Having variables as keys assures
# consistency.
# If it were lists, then a user could provide
# several functions for the same variable,
# which would make the update concurrent.
# wrong_substep_block = [
#     'prey_population': None,
#     'prey_population': None
# ]

### Parameters ###

# Prey Growth Rate
ALPHA = 2 / 3

# Prey Death Rate
BETA = 4 / 3

# Predator Growth Rate
DELTA = 1.0

# Predator Death Rate
GAMMA = 1.0

# Time interval
dt = 0.1

### State Update Functions ###

def prey_change(state: dict) -> float:
  x = state['prey_population']
  y = state['predator_population']
  dx = ALPHA * x - BETA * x * y
  value = max(x + dx * dt, 0)
  return value

def predator_change(state: dict) -> float:
  x = state['prey_population']
  y = state['predator_population']
  dy = DELTA * x * y - GAMMA * y
  value = max(y + dy * dt, 0)
  return value

def stochastic_process(state: dict) -> float:
  x = state['prey_population']
  value = x * (1 + random() / 1000)
  return value

### Partial State Update Block ###

## SUBs ##
# Represents intermediate state changes between t and t+1

substep_block_1 = {
    'prey_population': prey_change,
    'predator_population': predator_change,
}

substep_block_2 = {
    'prey_population': stochastic_process
}

# Represents a state change from t -> t+1
timestep_block = [
                  substep_block_1,
                  substep_block_2
]

# Simulation config

timestep_count = int(1e3)

### Simulation Execution ###

current_state = variables.copy()
results = []

# Iteration count: (T, S, f given S)
for timestep in tqdm(range(timestep_count)):

  # Iteration count: (S, f given S)
  for substep, substep_block  in enumerate(timestep_block):

    # We copy for making sure that the old state
    # which is history tracked is not changed
    new_state = current_state.copy()

    # Iteration count: f given S
    for variable, variable_update_function in substep_block.items():
      new_state[variable] = variable_update_function(current_state)

    # Append timestep and substep
    new_state.update(timestep=timestep, substep=substep)
    
    results.append(new_state)
    current_state = new_state


### Prepare results ###

import pandas as pd
df = pd.DataFrame(results)
df.head(10)

100%|██████████| 1000/1000 [00:00<00:00, 184811.81it/s]


,prey_population,predator_population,timestep,substep
0,0.500000,0.475000,0,0
1,0.500077,0.475000,0,1
2,0.501744,0.451254,1,0
3,0.501921,0.451254,1,1
4,0.505183,0.428778,2,0
5,0.505296,0.428778,2,1
6,0.510094,0.407566,3,0
7,0.510165,0.407566,3,1
8,0.516452,0.387602,4,0
9,0.516575,0.387602,4,1


In [22]:
import plotly.express as px

px.scatter(df,
           x='predator_population',
           y='prey_population',
           color='timestep')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('predator_population=%{x}<br>pr' ... '%{marker.color}<extra></extra>'),
              'legendgroup': '',
              'marker': {'color': array([  0,   0,   1, ..., 998, 999, 999]),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([4.75000000e-01, 4.75000000e-01, 4.51253672e-01, ..., 4.96272605e-08,
                          4.46645344e-08, 4.46645344e-08]),
              'xaxis': 'x',
              'y': array([0.5       , 0.5000773 , 0.50174423, ..., 0.        , 0.        ,
                          0.        ]),
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'timestep'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'predator_population'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'prey_population'}}}
})

)